In this script we process the raw data coming from the preprocessing to standarize the data into a working Anndata file for the rest of the analysis.


In [1]:
import scanpy as scp
import scipy.sparse as sprs
import pandas as pd
import warnings
import os
import numpy as np

warnings.filterwarnings("ignore")

# Gastruloid

In this step, we create the dataset for the AnnData format. 

We incorporate to the data the information coming from:

1. The raw count matrix.
2. The information coming from the flow cytometer sorting procedure.

In [2]:
genes = pd.read_csv("Data/Genes.csv")
cells = pd.read_csv("Data/Cells.csv")
m = sprs.load_npz("Data/count_matrix_total.npz")

#Adding plate information
#cells["Plate"] = 

statistics = pd.DataFrame()
statistics.loc[0,["Plates","Wells per plate","Total Wells"]] = [9,96,9*96]

In [14]:
(1715+13)/96

18.0

In [6]:
pd.DataFrame(np.array(m.todense()))

,0,1,2,3,4,5,6,7,8,9,...,53690,53691,53692,53693,53694,53695,53696,53697,53698,53699
0,235,0,11,0,0,0,0,0,82,14,...,0,0,0,0,0,0,0,0,0,0
1,272,0,102,0,0,0,314,0,138,0,...,0,0,0,0,0,0,0,0,0,0
2,236,0,8,0,0,0,0,0,98,102,...,0,0,0,0,0,0,0,0,0,0
3,494,0,134,0,0,0,0,0,64,12,...,0,0,0,0,0,0,0,0,0,0
4,226,0,372,0,0,0,176,0,60,72,...,0,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710,142,0,0,0,0,0,0,0,107,0,...,0,0,0,0,0,0,0,0,0,0
1711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1712,44,0,0,0,0,0,136,32,72,0,...,0,0,0,0,0,0,0,0,0,0
1713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Merging lanes <a name="Merging"></a>

In the preprocessing step, we aligned the files taking the different lanes as separated measures, however, these measures corrrespond to the cells. In this step, we unified the measures coming from different lanes measures coming from a unique well. 

In [3]:
dropIndex = [] # For dropping the index that are merged
statistics.loc[0,"singleLane"] = 0
for j,i in zip(cells[cells["Lane"]==1].index,cells[cells["Lane"]==1].values):
    
    # making the conditions to univocaly finding the cell with the other corresponding lane
    name = i[0]
    time = i[2]
    condition = i[3]
    
    cond1 = cells["Cell"] == name.replace("_1_","_2_")
    cond2 = cells["Time"] == time
    cond3 = cells["Lane"] == 2

    cell = cells[cond1.values*cond2.values*cond3.values]

    #Add the
    if cell.shape[0] == 1:
        m[j,:] += m[cell.index,:]
        dropIndex.append(cell.index[0])
    else:
        print(name.replace("_1_","_2_"), " at ", time,"_",condition, " doesn't exist. We only have one lane measure of this well.")
        statistics.loc[0,"singleLane"] += 1

#Drop merged cells
cells = cells.drop(dropIndex)

#Print cells that only have measures in second lane
for j,i in zip(cells[cells["Lane"]==2].index,cells[cells["Lane"]==2].values):
    name = i[0]
    time = i[1]
    condition = i[2]
    print(name.replace("_2_","_1_"), " at ", time,"_",condition, " doesn't exist. We only have one lane measure of this well.")
    statistics.loc[0,"singleLane"] += 1
#Change name of cells that are merged
cells["Cell"] = [i.replace("_1_","_").replace("_2_","_") for i in cells["Cell"].values]

#Retain only rows that correspond to merged cells
m = m[cells.index]

#Reset index of the cells
cells.reset_index(drop=True,inplace=True)

#Drop Lane information
cells.drop(columns="Lane",inplace=True)

HG5V7DRXY_2_IDT-DUI-NXT-341  at  216hr _ CD45  doesn't exist. We only have one lane measure of this well.


## Standarize the tags <a name="Standarize"></a>

We can check that in the raw data some tags have the same meaning but different nomenclature because of how they where given to the sequencing facility. We standarize the nomenclature.

1. kitScal and kitscal in Condition
2. empty and EMPTY in BiologicalSample

In [4]:
for i in cells.columns[1:]:
    print(i,": ", cells[i].unique())

Plate :  ['P1' 'P6' 'P7' 'P8' 'P9' 'P2' 'P4' 'P3' 'P5']
Time :  ['120hr' '192hr' '216hr' '144hr' '168hr']
Condition :  ['whole' 'CD45' 'kitscal' 'kitScal' 'CD41']
BiologicalSample :  ['P1' 'P2' 'empty' 'EMPTY' 'black']


In [5]:
cells.loc[cells["Condition"]=="kitscal","Condition"] = "kitScal"
cells.loc[cells["BiologicalSample"]=="EMPTY","BiologicalSample"] = "empty"

Additionally, we have a few cells marked with the tag black. Those correspond to measures did no have a valid measurement in the flow cytommetry system. This correspond to confirmed doublets by the flow cytometry sorting process as can be seen in the next section.

In [6]:
cells[cells["BiologicalSample"]=="black"]

,Cell,Plate,Time,Condition,BiologicalSample
583,HG72GDRXY_IDT-DUI-NXT-198,P3,144hr,CD41,black
598,HG72GDRXY_IDT-DUI-NXT-210,P3,144hr,CD41,black
611,HG72GDRXY_IDT-DUI-NXT-222,P3,144hr,CD41,black


## Adding Flow cytommetry measures to data <a name="Add_flow"></a>

For this experimental setup, the data has been sorted to enrich for rare cells. We add the measures of the cells to the available information of the system.

We check that there are wells that have flow information but there are not corresponding RNA sequenced measures.

Additionally we confirm that some wells were measured more than once indicating that the well contains doublets, this where marked already by the tag **black** in biological condition.

In [9]:
# List the sorting files by plates
plates = os.listdir("Data/SMARTseq")
#Add identifier flow cytometry file
cells["Flow Cytommetry File"] = np.NaN

#Counters
statistics.loc[0,"doublet"] = 0
statistics.loc[0,"noRNA"] = 0

for plate in plates:

    #Load table mapping well indexes to cell indexes
    table = pd.read_excel("Data/Mapping.xlsx",sheet_name=plate).set_index("X1").unstack()

    #Make mapping list of cells to plates
    mapping = {i[1]+str(i[0]):str(j).replace("_1_","_") for i,j in zip(table.index, table.values)}

    #List and go over the sorting files splited by plate
    for file_num,file_name in enumerate(os.listdir("Data/SMARTseq/"+plate)):

        #Load table of measures for the plate and
        data = pd.read_csv("Data/SMARTseq/"+plate+"/"+file_name,skiprows=9,decimal=",")
        data.set_index("Well",inplace=True)

        #Add information to the cell
        parameters = [i for i in data.columns if "All" in i and "Median" in i and "Time" not in i]
        parameters_clean = [i.split("Events ")[1].split(" Median")[0] for i in parameters]

        for i in data.index:
                if cells[cells["Cell"] == mapping[i]].shape[0] > 0:
                    if type(cells.loc[cells["Cell"] == mapping[i],"Flow Cytommetry File"].values[0]) == str:
                        print(cells.loc[cells["Cell"] == mapping[i]].index[0],cells.loc[cells["Cell"] == mapping[i],"Cell"].values[0], " is measured twice. Doublet.")
                        statistics.loc[0,"doublet"] += 1
                    else:
                        cells.loc[cells["Cell"] == mapping[i],"Flow Cytommetry File"] = plate+"_"+str(file_num)
                        for k,j in enumerate(parameters):
                            cells.loc[cells["Cell"] == mapping[i],parameters_clean[k]] = data.loc[i,j]
                else:
                    print(plate," ",i, " does not have scRNA data. Reading error.")
                    statistics.loc[0,"noRNA"] += 1


P2_144hr_whole   A11  does not have scRNA data. Reading error.
P9_216hr_CD45   B5  does not have scRNA data. Reading error.
P9_216hr_CD45   C5  does not have scRNA data. Reading error.
P9_216hr_CD45   D5  does not have scRNA data. Reading error.
P9_216hr_CD45   F5  does not have scRNA data. Reading error.
P9_216hr_CD45   H5  does not have scRNA data. Reading error.
583 HG72GDRXY_IDT-DUI-NXT-198  is measured twice. Doublet.
598 HG72GDRXY_IDT-DUI-NXT-210  is measured twice. Doublet.
611 HG72GDRXY_IDT-DUI-NXT-222  is measured twice. Doublet.


Finally, we check for wells that have transcription bu do not have any flow cytometry measure. Consistently, most of them correspond to wells called as empty, except for two samples that should be filled. These wells are wells that did not have any cell deposted in them.

In [10]:
statistics.loc[0,"empty"] = cells[cells["Flow Cytommetry File"].isna()*cells["BiologicalSample"]=="empty"].shape[0]
statistics.loc[0,"emptyWrong"] = cells[cells["Flow Cytommetry File"].isna()*np.array(cells["BiologicalSample"]!="empty")].shape[0]

cells[cells["Flow Cytommetry File"].isna()]

,Cell,Plate,Time,Condition,BiologicalSample,Flow Cytommetry File,FSC-A,FSC-H,FSC-W,SSC-A,SSC-H,SSC-W,GFP FLK-1 530/30 Blue-A,582/15 Y/G-A,PE Dazzle 594 CD41 610/20 Y/G-A,PECy7 Sca-1 780/60 Y/G-A,APC CD45 660/20 Red-A,APCCy7 cKIT 780/60 Red-A,Hoechst 58 450/40 Violet-A
89,HG3JFDRXY_IDT-DUI-NXT-282,P1,120hr,whole,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,HG3JFDRXY_IDT-DUI-NXT-288,P1,120hr,whole,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,HG5V7DRXY_IDT-DUI-NXT-186,P7,192hr,CD45,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,HG5V7DRXY_IDT-DUI-NXT-192,P7,192hr,CD45,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,HG5V7DRXY_IDT-DUI-NXT-282,P8,192hr,kitScal,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,HG5V7DRXY_IDT-DUI-NXT-288,P8,192hr,kitScal,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400,HG5V7DRXY_IDT-DUI-NXT-378,P9,216hr,CD45,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,HG5V7DRXY_IDT-DUI-NXT-384,P9,216hr,CD45,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465,HG5V7DRXY_IDT-DUI-NXT-90,P6,192hr,whole,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,HG5V7DRXY_IDT-DUI-NXT-96,P6,192hr,whole,empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Statistics, create Annotated data file and save <a name="Statistics"></a>

We will save the data of the files with non-zero RNA sample measures but we will keep those measures that contain doublets and empty well for now so they appear in the QC check.

In [11]:
percentage = 0
percentage += statistics.loc[0,"singleLane"]
percentage += statistics.loc[0,"doublet"]
percentage += statistics.loc[0,"noRNA"]
percentage += statistics.loc[0,"noRNA"]
percentage += statistics.loc[0,"noRNA"]
statistics.loc[0,"wells with bad samples"] = percentage
statistics.loc[0,"wells with good samples"] = statistics.loc[0,"Total Wells"]-percentage
statistics.loc[0,"% wells with bad samples"] = statistics.loc[0,"wells with bad samples"]*100/statistics.loc[0,"Total Wells"]
statistics.loc[0,"% wells with good samples"] = statistics.loc[0,"wells with good samples"]*100/statistics.loc[0,"Total Wells"]

In [12]:
statistics

,Plates,Wells per plate,Total Wells,singleLane,doublet,noRNA,empty,emptyWrong,wells with bad samples,wells with good samples,% wells with bad samples,% wells with good samples
0,9.0,96.0,864.0,1.0,3.0,6.0,17.0,2.0,22.0,842.0,2.546296,97.453704


In [13]:
#Create the Annotated data
adata = scp.AnnData(m,obs=cells,var=genes.set_index("gene_id"))

In [14]:
adata.write("Data/Raw.h5ad")

... storing 'Plate' as categorical
... storing 'Time' as categorical
... storing 'Condition' as categorical
... storing 'BiologicalSample' as categorical
... storing 'Flow Cytommetry File' as categorical
... storing 'gene_name' as categorical
